In [65]:
import import_ipynb
from LoadPicture import *

import numpy as np
import glob
import matplotlib.pyplot as plt

# Tensorflow imports
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications.resnet_v2 import ResNet50V2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

## Model parameters

In [66]:
batch_size = 32
class_names = ['FaceOnly', 'NoFace']

## Model callbacks

In [67]:
callbacks = [
    keras.callbacks.ModelCheckpoint(                        
                        filepath='../log/cps/{model.name}_cp_{epoch}', 
                        verbose=1, 
                        save_weights_only=False,
                        save_freq=5*batch_size) # Checkpoint is saved every five epochs
]

## Data parameters

In [68]:
image_height = 224
image_width = 224

## Creating the training dataset

In [69]:
face_training_ds = keras.utils.image_dataset_from_directory(
    "../data/training/",
    labels='inferred',
    label_mode='binary',
    class_names=class_names,
    color_mode='rgb',
    batch_size=batch_size,
    image_size=(image_height, image_width),
    shuffle=True,
    seed=None,
    validation_split=None,
    subset=None,
    interpolation='bilinear',
    follow_links=False,
    crop_to_aspect_ratio=True,
)

#face_training_ds.map(lambda x, y: (tf.image.resize(x, (image_height, image_width)), y))
face_training_ds.cache().batch(batch_size).prefetch(buffer_size=10)

Found 20 files belonging to 2 classes.


<PrefetchDataset element_spec=(TensorSpec(shape=(None, None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, None, 1), dtype=tf.float32, name=None))>

## Creating the validation dataset

In [70]:
validation_ds = []

## Creating the test dataset

In [71]:
test_ds = []

## Load the base model

In [72]:
# Loading either the ResNet50 architecture model or the previously saved model, and freeze it for transfer learning
base_model = ResNet50V2(include_top=False)
base_model.trainable = False

## Creating a new model

In [73]:
inputs = keras.Input(shape=(image_height, image_width, 3))

# Running base model in inference mode
x = base_model(inputs, training=False)

# Convert features of shape `base_model.output_shape[1:]` to vectors
x = keras.layers.GlobalAveragePooling2D()(x)

# Final layer for binary classification
outputs = keras.layers.Dense(1)(x)
model = keras.Model(inputs, outputs)

## Training the new model with new data

In [74]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[keras.metrics.BinaryAccuracy()])
history = model.fit(
            face_training_ds, 
            epochs=20, 
            callbacks=callbacks, 
            #validation_data=...
        )

Epoch 1/20
1/1 [==============================] - 10s 10s/step - loss: 3.4436 - binary_accuracy: 0.7500
Epoch 2/20
1/1 [==============================] - 2s 2s/step - loss: 5.6435 - binary_accuracy: 0.8500
Epoch 3/20
1/1 [==============================] - 2s 2s/step - loss: 5.5628 - binary_accuracy: 0.8500
Epoch 4/20
1/1 [==============================] - 2s 2s/step - loss: 2.6680 - binary_accuracy: 0.8500
Epoch 5/20
1/1 [==============================] - 2s 2s/step - loss: 0.9920 - binary_accuracy: 0.9000
Epoch 6/20
1/1 [==============================] - 2s 2s/step - loss: 4.1119 - binary_accuracy: 0.7000
Epoch 7/20
1/1 [==============================] - 2s 2s/step - loss: 2.8172 - binary_accuracy: 0.8000
Epoch 8/20
1/1 [==============================] - 2s 2s/step - loss: 0.5371 - binary_accuracy: 0.9500
Epoch 9/20
1/1 [==============================] - 2s 2s/step - loss: 0.5104 - binary_accuracy: 0.9000
Epoch 10/20
1/1 [==============================] - 2s 2s/step - loss: 1.2749 - b

## View Results

## Predict new images

In [75]:
# https://www.tensorflow.org/tutorials/images/classification

img = tf.keras.utils.load_img(
    '../data/test/FaceTest/nonFace_10.jpg', target_size=(image_height, image_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

#img_array = preprocess_input(x)

preds = model.predict(img_array)
score = tf.nn.softmax(preds[0])
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
#print('Predicted:', tf.keras.applications.imagenet_utils.decode_predictions(preds)[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

ValueError: Exception encountered when calling layer "tf.keras.backend.bias_add_4" (type TFOpLambda).

Dimensions must be equal, but are 2048 and 3 for '{{node tf.keras.backend.bias_add_4/BiasAdd}} = BiasAdd[T=DT_FLOAT, data_format="NHWC"](Placeholder, tf.keras.backend.bias_add_4/BiasAdd/bias)' with input shapes: [?,2048], [3].

Call arguments received by layer "tf.keras.backend.bias_add_4" (type TFOpLambda):
  • x=tf.Tensor(shape=(None, 2048), dtype=float32)
  • bias=tf.Tensor(shape=(3,), dtype=float32)
  • data_format='channels_last'